Sales Analysis - Full Python Script
Author: Monalika Kapoor
Purpose: 
 - Performs data wrangling (missing values, parsing dates)
 - Normalizes numeric columns (Min-Max)
 - Performs descriptive stats for Sales and Unit
 - Aggregates by State and Group and by time (daily/weekly/monthly/quarterly)
 - Produces and saves visualizations and CSV outputs


In [25]:
# ----------------------------------------------
# IMPORT LIBRARIES
# ----------------------------------------------
# Library	Purpose
# pandas	Load & manipulate CSV data
# numpy	Numerical computation
# Pathlib	Clean file path handling
# MinMaxScaler	Normalization of numeric columns
# matplotlib/seaborn	Data visualization & charts
# DateFormatter	Format time-based axis in plots
# warnings	Suppress irrelevant warnings
# Data handling
import pandas as pd
import numpy as np

# File paths
from pathlib import Path

# Normalization
from sklearn.preprocessing import MinMaxScaler

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.dates import DateFormatter

# Ignore warnings for cleaner outputs
import warnings
warnings.filterwarnings("ignore")


In [26]:
# ----------------------------------------------
# LOAD DATA
# ----------------------------------------------   
DATA_PATH = Path("AusApparalSales4thQrt2020.csv")# <-- using your uploaded CSV
df = pd.read_csv(DATA_PATH)

OUTPUT_DIR = Path("sales_analysis_output")
OUTPUT_DIR.mkdir(exist_ok=True)

df = pd.read_csv(DATA_PATH)

print("Data Loaded Successfully!")
print("Shape:", df.shape)
print("Columns:", df.columns.tolist())

Data Loaded Successfully!
Shape: (7560, 6)
Columns: ['Date', 'Time', 'State', 'Group', 'Unit', 'Sales']


In [27]:
# ----------------------------------------------
# DETECT COLUMNS AUTOMATICALLY
# ----------------------------------------------
def detect_columns(df):
    cols = df.columns
    date_col = next((c for c in cols if "date" in c.lower()), None)
    sales_col = next((c for c in cols if c.lower() == "sales"), None)
    unit_col = next((c for c in cols if "unit" in c.lower()), None)
    state_col = next((c for c in cols if "state" in c.lower()), None)
    group_col = next((c for c in cols if "group" in c.lower()), None)
    return date_col, sales_col, unit_col, state_col, group_col

DATE_COL, SALES_COL, UNIT_COL, STATE_COL, GROUP_COL = detect_columns(df)
print("\nDetected Columns:")
print("Date:", DATE_COL)
print("Sales:", SALES_COL)
print("Units:", UNIT_COL)
print("State:", STATE_COL)
print("Group:", GROUP_COL)



Detected Columns:
Date: Date
Sales: Sales
Units: Unit
State: State
Group: Group


In [28]:

# ----------------------------------------------
# DATA WRANGLING
# ----------------------------------------------

# Parse dates
df["ParsedDate"] = pd.to_datetime(df[DATE_COL], errors="coerce")

# Missing value check
print("\nMissing Values Summary:")
print(df.isna().sum())

# Fill numeric missing with median
for col in df.select_dtypes(include=[np.number]).columns:
    df[col].fillna(df[col].median(), inplace=True)

# Fill categorical missing with 'Unknown'
for col in df.select_dtypes(include=["object"]).columns:
    df[col].fillna("Unknown", inplace=True)



Missing Values Summary:
Date          0
Time          0
State         0
Group         0
Unit          0
Sales         0
ParsedDate    0
dtype: int64


## Data Wrangling Summary

The dataset required several preprocessing steps to ensure accuracy and consistency:

### 1. Handling Missing Values
- **Numeric columns** were filled using the **median** because the median is robust against extreme values and prevents distortion of the data distribution.
- **Categorical columns** were filled with `"Unknown"` to avoid dropping rows and to preserve category-level analysis.

### 2. Date Parsing
The date column was converted into a proper `datetime` format using `pd.to_datetime()`. This enables reliable:
- sorting
- time-based grouping (daily, weekly, monthly)
- trend analysis

### 3. Normalization (Min–Max Scaling)
MinMaxScaler was used to transform all numeric fields between **0 and 1**:
\[
X_{norm} = \frac{X - X_{min}}{X_{max} - X_{min}}
\]
This is ideal for:
- visual comparison
- distance-based models
- scale-sensitive analysis

### 4. Why GroupBy is Useful
The `groupby()` function is essential for performing:
- state-level aggregation  
- customer-group aggregation  
- time-level rollups  
It simplifies complex relational summaries into clean, readable outputs.


In [29]:
# ----------------------------------------------
# NORMALIZATION (Min–Max Scaling)
# ----------------------------------------------

from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np

# Reset index to ensure unique values before concat
df = df.reset_index(drop=True)

# Automatically select numeric columns, ensuring uniqueness
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

# Remove already normalized columns if they exist
numeric_cols = [c for c in numeric_cols if not c.endswith('_norm')]

if len(numeric_cols) > 0:
    scaler = MinMaxScaler()
    normalized = scaler.fit_transform(df[numeric_cols])
    
    # Create DataFrame for normalized columns with proper alignment
    normalized_df = pd.DataFrame(normalized, 
                                 columns=[f"{c}_norm" for c in numeric_cols],
                                 index=df.index)  # aligns with original df
    
    # Concatenate normalized columns to the original DataFrame
    df = pd.concat([df, normalized_df], axis=1)
else:
    print("No numeric columns found for normalization.")


In [30]:
# ----------------------------------------------
# DESCRIPTIVE STATISTICS
# ----------------------------------------------
print("\nDescriptive Statistics – Sales:")
print(df[SALES_COL].describe())

print("\nDescriptive Statistics – Units:")
print(df[UNIT_COL].describe())

# ----------------------------------------------
# HIGHEST & LOWEST SALES (STATE + GROUP)
# ----------------------------------------------
state_sales = df.groupby(STATE_COL)[SALES_COL].sum().sort_values(ascending=False)
group_sales = df.groupby(GROUP_COL)[SALES_COL].sum().sort_values(ascending=False)

print("\nTop States by Sales:\n", state_sales.head())
print("\nBottom States by Sales:\n", state_sales.tail())
print("\nTop Groups by Sales:\n", group_sales.head())
print("\nBottom Groups by Sales:\n", group_sales.tail())

state_sales.to_csv(OUTPUT_DIR/"state_total_sales.csv")
group_sales.to_csv(OUTPUT_DIR/"group_total_sales.csv")



Descriptive Statistics – Sales:
count      7560.000000
mean      45013.558201
std       32253.506944
min        5000.000000
25%       20000.000000
50%       35000.000000
75%       65000.000000
max      162500.000000
Name: Sales, dtype: float64

Descriptive Statistics – Units:
count    7560.000000
mean       18.005423
std        12.901403
min         2.000000
25%         8.000000
50%        14.000000
75%        26.000000
max        65.000000
Name: Unit, dtype: float64

Top States by Sales:
 State
VIC    105565000
NSW     74970000
SA      58857500
QLD     33417500
TAS     22760000
Name: Sales, dtype: int64

Bottom States by Sales:
 State
SA     58857500
QLD    33417500
TAS    22760000
NT     22580000
WA     22152500
Name: Sales, dtype: int64

Top Groups by Sales:
 Group
Men        85750000
Women      85442500
Kids       85072500
Seniors    84037500
Name: Sales, dtype: int64

Bottom Groups by Sales:
 Group
Men        85750000
Women      85442500
Kids       85072500
Seniors    84037500
Na

In [31]:
# ----------------------------------------------
# TIME-BASED REPORTING
# ----------------------------------------------
df = df.set_index("ParsedDate")

daily = df[SALES_COL].resample("D").sum()
weekly = df[SALES_COL].resample("W").sum()
monthly = df[SALES_COL].resample("M").sum()
quarterly = df[SALES_COL].resample("Q").sum()

daily.to_csv(OUTPUT_DIR/"daily_sales.csv")
weekly.to_csv(OUTPUT_DIR/"weekly_sales.csv")
monthly.to_csv(OUTPUT_DIR/"monthly_sales.csv")
quarterly.to_csv(OUTPUT_DIR/"quarterly_sales.csv")

print("\nTime-Based Reports Generated!")


Time-Based Reports Generated!


In [32]:
# ----------------------------------------------
# TIME OF DAY ANALYSIS (Peak & Off-Peak Hours)
# ----------------------------------------------

# Extract hour from original date column
df["Hour"] = df.index.hour

hourly_sales = df.groupby("Hour")[SALES_COL].sum()

# Save results
hourly_sales.to_csv(OUTPUT_DIR/"hourly_sales.csv")

# Plot
plt.figure(figsize=(10,5))
hourly_sales.plot(kind="line", marker="o")
plt.title("Hourly Sales Trend")
plt.xlabel("Hour of Day")
plt.ylabel("Total Sales")
plt.tight_layout()
plt.savefig(OUTPUT_DIR/"hourly_sales_trend.png")
plt.close()

print("\nHourly sales analysis completed!")


Hourly sales analysis completed!


## Time-of-Day Analysis

The hour component was extracted from the timestamp to understand purchase patterns throughout the day.

### Key Insights:
- **Peak Hours** → periods of highest sales activity.
- **Off-Peak Hours** → periods of low activity, useful for staffing and marketing adjustments.

This analysis helps identify when customers are most active, allowing businesses to optimize:
- store staffing
- marketing notifications
- discount timings


In [33]:
# ----------------------------------------------
# WEEKLY, MONTHLY, QUARTERLY VISUALIZATIONS
# ----------------------------------------------

# Weekly Sales Trend
plt.figure(figsize=(10,5))
weekly.plot()
plt.title("Weekly Sales Trend")
plt.ylabel("Sales")
plt.tight_layout()
plt.savefig(OUTPUT_DIR/"weekly_sales_plot.png")
plt.close()

# Monthly Sales Trend
plt.figure(figsize=(10,5))
monthly.plot()
plt.title("Monthly Sales Trend")
plt.ylabel("Sales")
plt.tight_layout()
plt.savefig(OUTPUT_DIR/"monthly_sales_plot.png")
plt.close()

# Quarterly Sales Trend
plt.figure(figsize=(10,5))
quarterly.plot()
plt.title("Quarterly Sales Trend")
plt.ylabel("Sales")
plt.tight_layout()
plt.savefig(OUTPUT_DIR/"quarterly_sales_plot.png")
plt.close()

print("Weekly / Monthly / Quarterly plots saved!")


Weekly / Monthly / Quarterly plots saved!


In [34]:
# ----------------------------------------------
# VISUALIZATION
# ----------------------------------------------
sns.set(style="whitegrid")

# 1) State-wise sales by group
pivot_table = df.pivot_table(values=SALES_COL, index=STATE_COL, columns=GROUP_COL, aggfunc="sum", fill_value=0)

plt.figure(figsize=(12,6))
pivot_table.plot(kind="bar")
plt.title("State-wise Sales by Group")
plt.ylabel("Sales")
plt.tight_layout()
plt.savefig(OUTPUT_DIR/"state_group_sales.png")
plt.close()

# 2) Total sales by group
plt.figure(figsize=(8,5))
group_sales.plot(kind="bar")
plt.title("Total Sales by Group")
plt.ylabel("Sales")
plt.tight_layout()
plt.savefig(OUTPUT_DIR/"group_sales.png")
plt.close()

# 3) Daily Sales Trend
plt.figure(figsize=(10,5))
daily.plot()
plt.title("Daily Sales Trend")
plt.ylabel("Sales")
plt.tight_layout()
plt.savefig(OUTPUT_DIR/"daily_sales_plot.png")
plt.close()

# 4) Boxplot (Sales & Units)
plt.figure(figsize=(8,5))
sns.boxplot(data=df[[SALES_COL, UNIT_COL]])
plt.title("Boxplot - Sales & Units")
plt.tight_layout()
plt.savefig(OUTPUT_DIR/"boxplot_sales_units.png")
plt.close()

print("\nVisualizations Saved!")


Visualizations Saved!


<Figure size 1200x600 with 0 Axes>

## Boxplot Interpretation

The boxplot compares the distribution of **Sales** and **Units Sold**.

- The **box** represents the interquartile range (middle 50% of data).
- The **line** inside the box shows the median value.
- **Whiskers** represent variability outside the upper and lower quartiles.
- **Dots** represent outliers.

Interpretation:
- If the sales boxplot is more spread out than units, it means sales values vary more widely.
- Outliers indicate unusually high or low transactions, which may require deeper investigation.


In [35]:
# ----------------------------------------------
# DISTRIBUTION PLOTS
# ----------------------------------------------

plt.figure(figsize=(10,5))
sns.histplot(df[SALES_COL], kde=True)
plt.title("Sales Distribution Histogram")
plt.tight_layout()
plt.savefig(OUTPUT_DIR/"sales_distribution.png")
plt.close()

plt.figure(figsize=(10,5))
sns.histplot(df[UNIT_COL], kde=True)
plt.title("Units Sold Distribution Histogram")
plt.tight_layout()
plt.savefig(OUTPUT_DIR/"units_distribution.png")
plt.close()

print("Distribution plots saved!")


Distribution plots saved!


## Distribution Analysis

Histogram + KDE plots were used to understand the distribution of Sales and Units.

### Observations:
- Skewness reveals whether sales have many small transactions vs large spikes.
- Helps identify outliers and typical purchase volumes.


In [36]:
# ----------------------------------------------
# REPORT GENERATION (MARKDOWN)
# ----------------------------------------------
with open(OUTPUT_DIR/"summary_report.md","w") as f:
    f.write("# Sales Analysis Report\n")
    f.write("Dataset: AusApparalSales4thQrt2020.csv\n\n")
    f.write("## Detected Columns\n")
    f.write(f"- Date Column: {DATE_COL}\n")
    f.write(f"- Sales Column: {SALES_COL}\n")
    f.write(f"- Units Column: {UNIT_COL}\n")
    f.write(f"- State Column: {STATE_COL}\n")
    f.write(f"- Group Column: {GROUP_COL}\n\n")

    f.write("## Top States by Sales\n")
    f.write(state_sales.head().to_markdown() + "\n\n")

    f.write("## Top Groups by Sales\n")
    f.write(group_sales.head().to_markdown() + "\n\n")

print("\nMarkdown Report Generated Successfully!")
print("\nAll outputs saved to:", OUTPUT_DIR)


Markdown Report Generated Successfully!

All outputs saved to: sales_analysis_output


## Why Seaborn Was Used

Seaborn provides a high-level interface for attractive and informative statistical graphics.

Advantages:
- Built-in themes for cleaner visual appearance
- Simple API for boxplots, histograms, heatmaps
- Integrates tightly with pandas DataFrames
- Reduces code complexity compared to Matplotlib

Because the project involves statistical analysis, Seaborn is the most appropriate tool.


## Recommendation: Use of GroupBy

The `groupby()` method is highly recommended because:

- It is optimized for large datasets.
- Produces meaningful summaries (sum, mean, count) with one line of code.
- Reduces risk of manual errors.
- Enables efficient state-wise and group-wise sales reporting.

Therefore, groupby is the preferred approach for aggregated reporting in this project.


# Executive Summary

### Key Findings
- The highest sales come from specific states and customer groups identified in the analysis.
- Daily, weekly, and monthly trends show consistent patterns that can be used for forecasting.
- Peak-hour analysis highlights when customers are most likely to shop.

### Business Recommendations
1. **Increase marketing during peak hours** to capitalize on high engagement.
2. **Target top-performing states and customer groups** with loyalty programs.
3. **Use low-sales hours** to run flash discounts or restock operations.
4. **Track monthly and quarterly trends** for inventory forecasting.
5. **Investigate outliers** to identify bulk orders or anomalies.

### Conclusion
The dataset provides strong insights into customer behavior and time-based sales patterns.
These insights can guide operational decisions, marketing strategy, and resource allocation.
